In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
#from spellchecker import SpellChecker
#import matplotlib.pyplot as plt
from collections import Counter #uso para n-gramas
#from wordcloud import WordCloud
import re # talvez regex vai ajudar na normalizaçao

1o heatmap já feito e no 2_keywords. O problema agora é o heatmap excluindo palavras do texto de apoio, que é o que farei aqui. SOMENTE o segundo heatmap!

In [2]:
csv_file_path = 'corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [3]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [4]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TÁ FUNCIONANDO AAAAAAAAAAAAAAAAA Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [6]:
# Tokenizando e retirando stopwords: importante para ver a frequencia das palavras

def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [39]:
df_va['tokenized_content'].sample(5)

1589    [Exmo, Prefeito, prefeitura, municipal, Belém,...
688     [Para, Prefeitura, Municipal, A, história, pov...
791     [Prezado, prefeito, Belém, ., jornal, “, Em, D...
818     [Belém, Junho2012, Srs, ., Jornal, Em, Dia, te...
511     [Belem, ,, 17, maio, 2012, ., Estimado, Prefei...
Name: tokenized_content, dtype: object

In [40]:
df_va.dtypes

file_name            object
content              object
nota                  int64
tokenized_content    object
dtype: object

Aqui abaixo que começa a parte de excluir o texto de apoio!!

In [7]:
# Specify the path to the .txt file
file_path = 'texto_tarefa4.txt'

# Read the content of the file
with open(file_path, "r", encoding="utf-8") as file:
    given_text = file.read()

# Print the content to verify
print(given_text)


Você é morador de Belém e está inconformado com a situação dos casarões históricos da cidade. Com base na matéria “Azulejos valiosos”, escreva uma carta aberta endereçada à prefeitura municipal, para ser publicada em jornais locais. Seu texto deverá explicar o problema e argumentar sobre a necessidade de se tomarem medidas imediatas para solucioná-lo.

Azulejos valiosos

Quatro casarões do século XIX são alvo de roubos e depredações em Belém.

A capital paraense já foi considerada uma das cidades brasileiras com maior variedade de azulejos, que coloriam as fachadas e o interior de residências. Boa parte deles foi importada da Europa, principalmente na virada do século XIX para o XX, auge da produção de borracha. Da década de 1970 para cá, no entanto, mais de 50% dos azulejos se perderam. Este ano, a situação parece ter se agravado. Desde fevereiro, pelo menos quatro casarões foram alvo de vandalismo. O assunto vem se espalhando pela capital paraense, e há quem suspeite de encomenda de 

In [8]:
# Pré-processamento- Texto de apoio

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TÁ FUNCIONANDO AAAAAAAAAAAAAAAAA Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

# Tokenizando e retirando stopwords: importante para ver a frequencia das palavras
def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

In [9]:
cleaned_text = clean_cp(given_text)
tokenized_text = tokenized_cp(cleaned_text)
tokenized_text

['morador',
 'belém',
 'inconformado',
 'situação',
 'casarões',
 'históricos',
 'cidade',
 'base',
 'matéria',
 'azulejos',
 'valiosos',
 'escreva',
 'carta',
 'aberta',
 'endereçada',
 'prefeitura',
 'municipal',
 'publicada',
 'jornais',
 'locais',
 'texto',
 'deverá',
 'explicar',
 'problema',
 'argumentar',
 'sobre',
 'necessidade',
 'tomarem',
 'medidas',
 'imediatas',
 'solucionálo',
 'azulejos',
 'valiosos',
 'quatro',
 'casarões',
 'século',
 'xix',
 'alvo',
 'roubos',
 'depredações',
 'belém',
 'capital',
 'paraense',
 'considerada',
 'cidades',
 'brasileiras',
 'maior',
 'variedade',
 'azulejos',
 'coloriam',
 'fachadas',
 'interior',
 'residências',
 'boa',
 'parte',
 'importada',
 'europa',
 'principalmente',
 'virada',
 'século',
 'xix',
 'xx',
 'auge',
 'produção',
 'borracha',
 'década',
 'cá',
 'entanto',
 'azulejos',
 'perderam',
 'ano',
 'situação',
 'parece',
 'ter',
 'agravado',
 'desde',
 'fevereiro',
 'menos',
 'quatro',
 'casarões',
 'alvo',
 'vandalismo',
 'ass

In [10]:
# Retirando tokens do texto de apoio do conjunto de tokens dos textos dos candidatos

# Ensure tokenized_text is a set for efficient subtraction
tokenized_text_set = set(tokenized_text) # Using a set for tokenized_text ensures efficient lookups when checking if a token should be excluded.

# Subtract tokenized_text from each row in 'tokenized_content'
df_va['filtered_content'] = df_va['tokenized_content'].apply(
    lambda tokens: [token for token in tokens if token not in tokenized_text_set]
)

# Display the updated DataFrame
df_va.head()


,file_name,content,nota,tokenized_content,filtered_content
0,20152t4p1166n0r.docx,é inaceitável e constrangedor a violência que ...,0,"[inaceitável, constrangedor, violência, existe...","[inaceitável, constrangedor, violência, existe..."
1,20152t4p1305n0r.docx,a capital paraense já foi considerada uma das ...,0,"[capital, paraense, considerada, cidades, bras...","[enterior, importadas, palacetes, tomem, enqua..."
2,20152t4p1477n0r.docx,a prefeitura municipal respeitosa sou a public...,0,"[prefeitura, municipal, respeitosa, publicidad...","[respeitosa, publicidade, vou, explicálo, impo..."
3,20152t4p1506n0r.docx,data de bibiana c terra para prefeitura munici...,0,"[data, bibiana, c, terra, prefeitura, municipa...","[data, bibiana, c, terra, gritos, socorro, pre..."
4,20152t4p1512n0r.docx,desculpe eu não conhecome a matéria azulejos v...,0,"[desculpe, conhecome, matéria, azulejos, valio...","[desculpe, conhecome, acho]"


In [11]:
# Contando o número de tokens SEM STOPWORDS (diferente do typestokenttr.ipynb) para cada texto: usarei isso para o gráfico 2, que demonstra o número mínimo e número máximo de tokens

# Contando tokens sem considerar a nota
df_va['num_tokens'] = df_va['filtered_content'].apply(len)

# Agrupando de acordo com cada nota
qtde_tokens_nota = df_va.groupby('nota')['num_tokens'].sum()

In [12]:
# Agrupando a contagem de tokens de acordo com cada nota MANTER
nota_token_counts = (
    df_va.groupby('nota')['filtered_content']
    .apply(lambda texts: Counter([token for text in texts for token in text]))
)

# Convertendo cada token e contagem em um dataframe
df_frequency = nota_token_counts.reset_index()
df_frequency.columns = ['nota', 'token', 'token_frequency']
sorted_tokens = df_frequency.sort_values('token_frequency', ascending=False)

In [13]:
df_words_ta = pd.DataFrame()

In [14]:
# Agrupando a contagem de tokens por nota de forma a mostrar os 15 mais comuns para cada nota ( 15 * 6 = 90 linhas portanto )
top_tokens_per_grade = (
    sorted_tokens.groupby('nota')
    .head(100)  # Assim mostra os 1000 tokens mais comuns. Já vai ficar péssimo na visualização entao mantenho esse head e insiro outro abaixo.
    .reset_index(drop=True)
)
df_words_ta = top_tokens_per_grade

In [ ]:
# df_words_sorted = (df_words.sort_values(by=['nota', 'token_frequency'], ascending=[True, False]))

In [15]:
import plotly.graph_objects as go ### ESSE AQUI QUE VOU USAR NO STREAMLIT ###
import pandas as pd

# Pivot the data for heatmap
heatmap_data = df_words_ta.pivot(index='token', columns='nota', values='token_frequency').fillna(0).head(20)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência", titlefont=dict(size=14), tickfont=dict(size=12)),  # Legend title and font size
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "<span style='font-size:14px'><b>Token</b>: %{y}<br>" +  # Tooltip with larger font
        "<b>Nota</b>: %{x}<br>" +
        "<b>Frequência</b>: %{z}</span><extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white", size=14),  # Update general font size
    height=800,  # Adjust height
    width=1000,   # Keep width consistent
)

# Update y-axis to hide tokens
fig.update_yaxes(
    title="",  # No y-axis title
    showticklabels=False,  # Hide tokens from y-axis
)

# Update x-axis for better styling
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()
